We compare the outputs of the models using the metrics in notebook 2. 

In [35]:
## load in the results and the truth: 


import pandas as pd
import numpy as np
test = pd.read_csv("data/test.txt", header=None, sep=" ")
y_test = test[0]



In [36]:
pred_lm6 = pd.read_csv("data/pred_lm6.txt", header=None, sep=" ")[1]
pred_lm2 = pd.read_csv("data/pred_lm2.txt", header=None, sep=" ")[1]
pred_lr6 = pd.read_csv("data/pred_lr6.txt", header=None, sep=" ")[1]
pred_lr2 = pd.read_csv("data/pred_lr2.txt", header=None, sep=" ")[1]

In [37]:
def bubble_sort(vals_list):
    swaps = 0
    n=len(vals_list)
    sorted = 0
    while sorted == 0:
        swaps_this_pass = 0
        for i in range(0, n-1):

            if vals_list[i]>vals_list[i+1]:
                vals_list[i], vals_list[i+1] = vals_list[i+1], vals_list[i]
                swaps_this_pass = swaps_this_pass + 1

        if swaps_this_pass==0:
            sorted=1
        swaps = swaps + swaps_this_pass        
        n = n-1 #ith pass of bubble sort puts the nth value in order.
            #so there is no need to consider this. 
    return(swaps)

In [38]:
def bubble_swaps(group):
    truth_by_pred = group.sort_values(by = "predicted").truth.tolist()
    swaps = bubble_sort(truth_by_pred)
    return swaps

In [39]:
import math
""" this returns 0 if all of the ordered data is undesirable"""
def ndcg_p(ordered_data, p):
    """normalised discounted cumulative gain"""
    if sum(ordered_data)==0:
        return 0
    else:
        indexloop = range(0, p)
        DCG_p = 0
        for index in indexloop:
            current_ratio=(2**(ordered_data[index])-1)*(math.log((float(index)+2), 2)**(-1))
            DCG_p = DCG_p + current_ratio
        sorted_data= sorted(ordered_data,reverse=True)
        n = len(ordered_data)
        indexloop = range(0, n)
        iDCG_p = 0
        for index in indexloop:
            current_ratio=(2**(sorted_data[index])-1)*((math.log((index+2), 2))**(-1))
            iDCG_p = iDCG_p + current_ratio
        return(DCG_p/iDCG_p)
    
    
def compute_at_p(ordered_data, p): 
    """
    returns -1 if number of items in list is less than p
    else returns ndcg@p
    """
    n = len(ordered_data)
    if n<p:
        result = np.nan
    else:
        result = ndcg_p(ordered_data, p)
    
    return result


def sort_ndcg(unordered_group, p):
    ordered_group = unordered_group.sort_values(by = "predicted").truth.tolist()
    return compute_at_p(ordered_group, p)



In [40]:
from sklearn.metrics import mean_squared_error

print(mean_squared_error(pred_lm6, y_test))
print(mean_squared_error(pred_lm2, y_test))
print(mean_squared_error(pred_lr6, y_test))
print(mean_squared_error(pred_lr2, y_test))





0.7349503461764648
0.6996125253331467
0.6611331099722424
0.6856596521689255


In [41]:
preds_df_lm6 = pd.DataFrame({'qid': test[1], 'truth' :y_test, 'predicted' : pred_lm6 })

In [42]:
groups_lm6 = preds_df_lm6.groupby('qid')

In [43]:
groups_lm6

In [44]:
pairwise_lm6 = groups_lm6.apply(bubble_swaps)

In [45]:
np.mean(pairwise_lm6)

2190.582

In [46]:
ndcg_lm6_100 = groups_lm6.apply(sort_ndcg, p=100)

In [47]:
np.nanmean(ndcg_lm6_100)

0.3536186971513519

In [48]:
preds_df_lm2 = pd.DataFrame({'qid': test[1], 'truth' :y_test, 'predicted' : pred_lm2 })
groups_lm2 = preds_df_lm2.groupby('qid')
pairwise_lm2 = groups_lm2.apply(bubble_swaps)
print(np.mean(pairwise_lm2))
ndcg_lm2_100 = groups_lm2.apply(sort_ndcg, p=100)
print(np.nanmean(ndcg_lm2_100))

2350.1675
0.38323430363485084


In [53]:
ndcg_lm2_500 = groups_lm2.apply(sort_ndcg, p=500)
print(np.nanmean(ndcg_lm2_500))

0.7116672567519972


In [52]:
ndcg_lm6_500 = groups_lm6.apply(sort_ndcg, p=500)
print(np.nanmean(ndcg_lm6_500))

0.707635785366559
